# 성적 + 가치관 기반 추천시스템

- 성적정보와 가치관정보를 모두 코사인유사도를 기반으로 분석하여 적절한 팀메이트 추천
- 성적정보 : 총점은 비슷하고, 영역별 점수는 달라야함
- 가치관정보 : 영역별 점수가 최대한 유사해야함

In [9]:
import numpy as np
import pandas as pd

# 점수 0-10점으로 랜덤 생성
# 인원 수 160명
user_grade = pd.DataFrame(np.random.randint(1,11,size=(160,4)) , index=range(1,161), columns=['데이터분석','머신러닝','웹','디자인'])
# 각 유저(사용자)별 네 점수 총합
user_grade_sum = user_grade.sum(axis=1)

display(user_grade)
display(user_grade_sum)

,데이터분석,머신러닝,웹,디자인
1,7,8,9,5
2,6,6,6,8
3,9,5,2,3
4,6,3,10,4
5,8,8,2,7
...,...,...,...,...
156,3,5,4,3
157,6,6,10,9
158,9,8,3,4
159,7,8,1,10


1      29
2      26
3      19
4      23
5      25
       ..
156    15
157    31
158    24
159    26
160    16
Length: 160, dtype: int64

In [10]:
# 가치관점수 : 
# 디테일 <-> 속도 : 디테일을 중요시하는지 속도를 중요시하는지
# 과정 <-> 결과 : 과정을 중요시하는지 결과를 중요시하는지
# 창업 <-> 취업 : 창업을 꿈꾸는지 취업을 꿈꾸는지
# 아침 <-> 밤 : 아침형인간인지 밤형 인간인지

# 점수 0-100점으로 랜덤 생성
user_style = pd.DataFrame(np.random.randint(1,101,size=(160,4)) , index=range(1,161), columns=['속도','결과','취업','밤'])
user_style['디테일'] = 100 - user_style['속도']
user_style['과정'] = 100 - user_style['결과']
user_style['창업'] = 100 - user_style['취업']
user_style['아침'] = 100 - user_style['밤']
display(user_style)

,속도,결과,취업,밤,디테일,과정,창업,아침
1,36,52,75,67,64,48,25,33
2,98,66,72,86,2,34,28,14
3,56,79,66,13,44,21,34,87
4,23,42,96,93,77,58,4,7
5,43,26,84,47,57,74,16,53
...,...,...,...,...,...,...,...,...
156,55,98,36,80,45,2,64,20
157,95,84,13,57,5,16,87,43
158,9,90,59,85,91,10,41,15
159,18,100,10,59,82,0,90,41


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

# 유저와 유저 간의 유사도
# 성적 기반 유사도 (낮을수록 추천도 상승)
user_based_collab = cosine_similarity(user_grade, user_grade)
print(user_based_collab)

[[1.         0.94805065 0.84244852 ... 0.90178442 0.79451025 0.81102923]
 [0.94805065 1.         0.83877126 ... 0.88890409 0.91736371 0.94968846]
 [0.84244852 0.83877126 1.         ... 0.97727516 0.84596696 0.75770836]
 ...
 [0.90178442 0.88890409 0.97727516 ... 1.         0.891287   0.77289099]
 [0.79451025 0.91736371 0.84596696 ... 0.891287   1.         0.90559012]
 [0.81102923 0.94968846 0.75770836 ... 0.77289099 0.90559012 1.        ]]


In [14]:
# 1번 입장에서 진행, 각 점수에서 1번점수를 빼고 절대값을 씌워 1번 사용자의 총합과 얼마나 차이나는지 결정
# 0.1을 곱한 이유? 그대로 사용하면 총합점수차 지표의 영향력이 너무 커진다. 
grade_subs = np.abs(user_grade_sum - user_grade_sum[1]) 

display(grade_subs)

1       0
2       3
3      10
4       6
5       4
       ..
156    14
157     2
158     5
159     3
160    13
Length: 160, dtype: int64

In [15]:
# 가치관에 대한 유사도
# 높을 수록 추천도가 높아진다.

user_based_collab_style = cosine_similarity(user_style, user_style)
print(user_based_collab_style)

[[1.         0.83346722 0.82671706 ... 0.90114911 0.74625645 0.8196059 ]
 [0.83346722 1.         0.72982567 ... 0.70040926 0.58972648 0.90534157]
 [0.82671706 0.72982567 1.         ... 0.72295381 0.74596996 0.66385705]
 ...
 [0.90114911 0.70040926 0.72295381 ... 1.         0.89083347 0.58294845]
 [0.74625645 0.58972648 0.74596996 ... 0.89083347 1.         0.51537356]
 [0.8196059  0.90534157 0.66385705 ... 0.58294845 0.51537356 1.        ]]


In [16]:
# 높은 추천도를 가지려면?
# user_based_collab(성적영역 유사도)가 낮아야함 (각 영역별 수치가 최대한 달라야 하므로)
# grade_subs(총점수차)가 낮아야함. (차이가 적어야 비슷한 실력이므로)
# user_based_collab_style(가치관 유사도)는 높아야함

# 아래와 같은 수식을 최종 지표로 활용한다.
# 해당 최종 지표가 낮을수록 추천도는 높다
evaluation_value = user_based_collab[0] + grade_subs - user_based_collab_style[0]

evaluation_value

1     -4.440892e-16
2      3.114583e+00
3      1.001573e+01
4      5.981340e+00
5      3.921172e+00
           ...     
156    1.415141e+01
157    2.278209e+00
158    5.000635e+00
159    3.048254e+00
160    1.299142e+01
Length: 160, dtype: float64

In [17]:
# 해당 지표가 낮은 순서대로 index 뽑아옴
top10 = evaluation_value.sort_values()[:10].index.tolist()

# 추천되는 인원의 인덱스 리스트
top10

[65, 1, 34, 149, 151, 147, 87, 126, 134, 25]

In [21]:
# 주어진 유저가 어떤 스타일인지(가치관인지) 확인하는 함수

def checkStyle(user):
    style = []
    if user['디테일'] > user['속도']:
        style.append('디테일')
    else:
        style.append('속도')
    if user['과정'] > user['결과']:
        style.append('과정')
    else:
        style.append('결과')
    if user['창업'] > user['취업']:
        style.append('창업')
    else:
        style.append('취업')
    if user['아침'] > user['밤']:
        style.append('아침')
    else:
        style.append('밤')
    return style

In [25]:
# 위와 같은 원리로 인풋 값에 대해 추천시스템 작동
# 가치관이 비슷하고,
# 성적총점은 비슷하지만 잘하는 영역은 다른 유저를 추천해준다.

input_num = int(input()) - 1
grade_subs = np.abs(user_grade_sum - user_grade_sum[input_num + 1]) * 0.1
evaluation_value = user_based_collab[input_num] + grade_subs - user_based_collab_style[input_num]
top10 = evaluation_value.sort_values()[:10].index.tolist()

print(f"<나의 정보>")
print()
print(f"{checkStyle(user_style.iloc[input_num])}")
print(f"데이터분석 : {user_grade.iloc[input_num]['데이터분석']}, 머신러닝 : {user_grade.iloc[input_num]['머신러닝']}, 웹 : {user_grade.iloc[input_num]['웹']}, 디자인 : {user_grade.iloc[input_num]['디자인']}, 총합 : {user_grade_sum[input_num + 1]}")
print(f"강점 : {user_grade.iloc[input_num].sort_values(ascending=False)[:2].index.tolist()}")
print()

print("================================")
for idx, value in enumerate(top10):
    value = value - 1
    print()
    print(f"<추천 {idx + 1}순위 : {value}번째 학생>")
    print()
    print(f"{checkStyle(user_style.iloc[value])}")
    print(f"데이터분석 : {user_grade.iloc[value]['데이터분석']}, 머신러닝 : {user_grade.iloc[value]['머신러닝']}, 웹 : {user_grade.iloc[value]['웹']}, 디자인 : {user_grade.iloc[value]['디자인']}, 총합 : {user_grade_sum[value + 1]}")
    print(f"강점 : {user_grade.iloc[value].sort_values(ascending=False)[:2].index.tolist()}")
    print()
    print("=========================")

<나의 정보>

['디테일', '과정', '취업', '아침']
데이터분석 : 6, 머신러닝 : 1, 웹 : 3, 디자인 : 9, 총합 : 19
강점 : ['디자인', '데이터분석']


<추천 1순위 : 78번째 학생>

['속도', '과정', '취업', '아침']
데이터분석 : 9, 머신러닝 : 3, 웹 : 6, 디자인 : 1, 총합 : 19
강점 : ['데이터분석', '웹']


<추천 2순위 : 84번째 학생>

['속도', '과정', '취업', '아침']
데이터분석 : 5, 머신러닝 : 9, 웹 : 1, 디자인 : 3, 총합 : 18
강점 : ['머신러닝', '데이터분석']


<추천 3순위 : 106번째 학생>

['디테일', '과정', '취업', '밤']
데이터분석 : 5, 머신러닝 : 10, 웹 : 2, 디자인 : 3, 총합 : 20
강점 : ['머신러닝', '데이터분석']


<추천 4순위 : 23번째 학생>

['디테일', '과정', '취업', '밤']
데이터분석 : 7, 머신러닝 : 7, 웹 : 3, 디자인 : 2, 총합 : 19
강점 : ['머신러닝', '데이터분석']


<추천 5순위 : 82번째 학생>

['속도', '결과', '취업', '아침']
데이터분석 : 10, 머신러닝 : 7, 웹 : 1, 디자인 : 1, 총합 : 19
강점 : ['데이터분석', '머신러닝']


<추천 6순위 : 137번째 학생>

['디테일', '과정', '취업', '밤']
데이터분석 : 4, 머신러닝 : 1, 웹 : 10, 디자인 : 4, 총합 : 19
강점 : ['웹', '디자인']


<추천 7순위 : 53번째 학생>

['속도', '결과', '취업', '아침']
데이터분석 : 6, 머신러닝 : 2, 웹 : 8, 디자인 : 2, 총합 : 18
강점 : ['웹', '데이터분석']


<추천 8순위 : 59번째 학생>

['속도', '과정', '취업', '밤']
데이터분석 : 3, 머신러닝 : 7, 웹 : 9, 디자인 : 2, 총합 : 21
강점 : ['웹